Creation of Map for Tavels made only in Beijing City

In [ ]:
import osmnx as ox
from shapely.geometry import Point,box
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap
import time

In [ ]:
start_time = time.time()


# geocode Beijing city to retrieve its boundary polygon
city = ox.geocode_to_gdf("Beijing, China").iloc[0]
boundary_polygon = city.geometry
# create a geopandas GeoDataFrame for the boundary polygon
boundary = gpd.GeoDataFrame(geometry=[boundary_polygon])

print(f"Total runtime for the function: {end_time - start_time:.3f} seconds.")

Total runtime for the function: 119.245 seconds.


In [ ]:
m

In [ ]:
start_timer = time.time()

# geocode Beijing city to retrieve its boundary polygon
city = ox.geocode_to_gdf("Beijing, China").iloc[0]
boundary_polygon = city.geometry
# create a geopandas GeoDataFrame for the boundary polygon
boundary = gpd.GeoDataFrame(geometry=[boundary_polygon])

end_timer = time.time()
print(f"Total runtime for the function: {end_timer - start_timer:.3f} seconds.")

Total runtime for the function: 0.216 seconds.


Using chunks create a geodataframe for every chunk and filter rows which are within boundary of beijing city 

In [ ]:
start_timer = time.time()
chunk_size = 100000

results = []
i=0
# read the data in chunks and process each chunk
for chunk in pd.read_csv("/content/drive/MyDrive/Colab_files/combined_trajectories.csv", chunksize=chunk_size):
    
    #create a geodataframe for chunk and check if the rows in dataframe are within boundary
    gdf = gpd.GeoDataFrame(chunk, geometry=gpd.points_from_xy(chunk.longitude, chunk.latitude))
    result = gpd.sjoin(gdf, boundary,predicate='within')
    results.append(result)

City_df = pd.concat(results, ignore_index=True)
City_df = City_df.drop('index_right', axis=1)
end_timer = time.time()
print(f"Total runtime : {end_timer - start_timer:.3f} seconds.")

Total runtime for the function: 117.596 seconds.


Filter the data based on what we want to graph.
I have used filtered data to create heat map for trips between 17:00:00 to 18:00:00 on 2023-10-23 

In [ ]:

City_df=City_df[(City_df['time']>'21:00:00') & (City_df['time']<'22:00:00')]
City_df.dropna()

Create sqaure grids of size 0.01 degree using the minimum and maximum longitude amd latitudes in the dataset. It also calculates centre of the grid. the centre of grid and grid boundary points(four corners) in grid dataframe

In [ ]:
# Create grid polygon layer
start_timer = time.time()
minx, miny, maxx, maxy = City_df.total_bounds
cell_size = 0.01
grid = []
centres=[]
for x in range(int(minx/cell_size), int(maxx/cell_size)+1):
    for y in range(int(miny/cell_size), int(maxy/cell_size)+1):
        grid.append(box(x*cell_size, y*cell_size, (x+1)*cell_size, (y+1)*cell_size))
        centres.append(Point((x*cell_size+(x+1)*cell_size)/2, (y*cell_size+(y+1)*cell_size)/2))
centres_df=pd.DataFrame({"centres":centres})
grid = gpd.GeoDataFrame(centres_df,geometry=grid)
end_timer = time.time()
print(f"Total runtime for the function: {end_timer - start_timer:.3f} seconds.")

performs join on grid dataframe to find which grid each row falls. The resulting dataframe is point_grid_df.

In [ ]:
point_grid_df = gpd.sjoin(City_df, grid, predicate='within')

grid_count_df get the count of unique individual_id from the point_grid_df(total rows) by grouping using index_right. It then joins with grid to add centres and their . It renames index_right to grid_id. It finally sets the "centres" column as the geometry column of the geopandas DataFrame so hat the "centres" column will be treated as the spatial information for each row of the DataFrame when performing spatial operations or visualizations. 

In [ ]:
start_timer = time.time()
grid_count_df = point_grid_df.groupby('index_right')['individual_id'].nunique().reset_index()
grid_count_df = grid_count_df.join(grid, on='index_right')
grid_count_df = grid_count_df.rename(columns={'index_right': 'grid_id','individual_id': 'count'})
grid_count_df=grid_count_df.set_geometry('centres')
end_timer = time.time()
print(f"Total runtime for the function: {end_timer - start_timer:.3f} seconds.")

Total runtime for the function: 0.047 seconds.


Create heat map using folium considering count as weight

In [ ]:
start_timer = time.time()
points = [(p.y, p.x) for p in grid_count_df['centres']]

# Create a list of tuples with the point coordinates and the weight (count). The weight(count) will be used to increase the colour based on density(count)
heat_data = [(p[0], p[1], c) for p, c in zip(points, grid_count_df['count'])]

# Create a folium map centered at the mean coordinates of the points
m = folium.Map(location=[grid_count_df['centres'].y.mean(), grid_count_df['centres'].x.mean()], zoom_start=10)

# Add the heatmap layer to the map
HeatMap(heat_data).add_to(m)
end_timer = time.time()
print(f"Total runtime for the function: {end_timer - start_timer:.3f} seconds.")

Total runtime for the function: 0.047 seconds.


In [ ]:
m